In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval

In [2]:
metal = 'Cu'

In [3]:
# Metal charge by cell2mol or BVS (only cases that cell reconstruction finished successfully)
df_bvs = pd.read_csv(f"BVS_result_{metal}.txt",delimiter="\t") 

# charge extracted from cif files
df_cif = pd.read_csv(f"cif_charge_{metal}.txt",delimiter="\t") 

In [4]:
print(len(df_bvs))
print(len(df_bvs['refcode'].unique()))
df_bvs['metal'].unique()

11361
11361


array(['Cu'], dtype=object)

In [5]:
print(len(df_cif))
print(len(df_cif['refcode'].unique()))
df_cif['metal'].unique()

12863
12863


array(['Cu'], dtype=object)

In [6]:
df = df_bvs.merge(df_cif, on='refcode', how='left')
df = df.drop(['metal_y', 'unique_indices'], axis=1)
df = df.rename(columns={'metal_x': 'metal'})
df['dictionary'] = df.dictionary.apply(lambda x: literal_eval(str(x)))
cols = ['refcode', 'metal', 'charge_cif', 'charge_cell2mol', 'charge_BVS', 'dictionary']
df = df[cols] 
df = df.rename(columns={'charge_cif': 'Cif', 'charge_cell2mol' :'cell2mol', 'charge_BVS' : 'BVS'})

In [7]:
thre_max = 0.5

df['bvs_min'] = df.dictionary.apply(lambda x: sorted(x, key=x.get)[0])
df['min'] = df.dictionary.apply(lambda x: x[sorted(x, key=x.get)[0]])
df['bvs_second_min'] = df.dictionary.apply(lambda x: sorted(x, key=x.get)[1])
df['second_min'] = df.dictionary.apply(lambda x: x[sorted(x, key=x.get)[1]])

condition = (lambda x: 
 x[sorted(x, key=x.get)[1]]- x[sorted(x, key=x.get)[0]]  # (second_min_delta - min_delta)
 if (x[sorted(x, key=x.get)[0]] <= thre_max and x[sorted(x, key=x.get)[1]] <= thre_max) 
 else None)
df['diff'] = df.dictionary.apply(condition)

In [8]:
# pd.set_option('display.max_colwidth', None)

In [9]:
df

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.001990,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.284670
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
...,...,...,...,...,...,...,...,...,...,...,...
11356,ZUYWAU,Cu,2,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2,0.000069,1,0.256987,0.256918
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11358,ZUZKOX,Cu,1,1,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.000000,2,9999.000000,NaN
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


In [10]:
print(len(df['refcode'].unique()))
df['metal'].unique()

11361


array(['Cu'], dtype=object)

# cell2mol 

In [11]:
print(f"Total cases : {len(df)}")

# cell2mol works
print(f"\ncell2mol works : {len(df[df['cell2mol'] != 9999])}")
print(f"cell2mol works, Same metal charge by cell2mol and cif : {len( df[(df['cell2mol'] == df['Cif']) & (df['cell2mol'] != 9999)] )}")
print(f"cell2mol works, Different metal charge by cell2mol and cif : : {len( df[(df['cell2mol'] != df['Cif']) & (df['cell2mol'] != 9999) ] )}")

# cell2mol doesn't work ==> cell2mol code : 9999
print(f"\ncell2mol doesn't work : {len(df[df['cell2mol'] == 9999])}") 

Total cases : 11361

cell2mol works : 9978
cell2mol works, Same metal charge by cell2mol and cif : 9791
cell2mol works, Different metal charge by cell2mol and cif : : 187

cell2mol doesn't work : 1383


In [12]:
df[df['cell2mol'] != 9999]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.001990,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.284670
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
...,...,...,...,...,...,...,...,...,...,...,...
11350,ZUXYID,Cu,1,1,7777,"{1: 0.060687, 2: 0.130107, 3: 9999}",1,0.060687,2,0.130107,0.069420
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11358,ZUZKOX,Cu,1,1,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.000000,2,9999.000000,NaN
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


In [13]:
df[(df['cell2mol'] == df['Cif']) & (df['cell2mol'] != 9999)]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.001990,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.284670
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
...,...,...,...,...,...,...,...,...,...,...,...
11350,ZUXYID,Cu,1,1,7777,"{1: 0.060687, 2: 0.130107, 3: 9999}",1,0.060687,2,0.130107,0.069420
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11358,ZUZKOX,Cu,1,1,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.000000,2,9999.000000,NaN
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


In [14]:
df[(df['cell2mol'] != df['Cif']) & (df['cell2mol'] != 9999)]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
177,AKIYUO,Cu,3,2,7777,"{1: 0.257117, 2: 5.8e-05, 3: 0.49775}",2,0.000058,1,0.257117,0.257059
251,ANICEI,Cu,3,1,7777,"{1: 0.35015, 2: 0.001642, 3: 0.185694}",2,0.001642,3,0.185694,0.184052
578,BASLOW,Cu,2,1,7777,"{1: 0.375281, 2: 0.03511, 3: 0.396453}",2,0.035110,1,0.375281,0.340171
585,BATWEB,Cu,2,1,7777,"{1: 0.294117, 2: 0.000284, 3: 0.521945}",2,0.000284,1,0.294117,0.293833
586,BATWOL,Cu,2,1,7777,"{1: 0.275833, 2: 4e-06, 3: 0.42835}",2,0.000004,1,0.275833,0.275829
...,...,...,...,...,...,...,...,...,...,...,...
10878,YINBAY,Cu,2,1,7777,"{1: 0.434864, 2: 0.085897, 3: 0.107196}",2,0.085897,3,0.107196,0.021299
11106,ZAWHEM,Cu,1,2,7777,"{1: 0.00318, 2: 0.193075, 3: 1.168491}",1,0.003180,2,0.193075,0.189895
11163,ZESSAT,Cu,2,1,7777,"{1: 0.204365, 2: 0.000996, 3: 0.593918}",2,0.000996,1,0.204365,0.203369
11313,ZUBJOV,Cu,2,1,7777,"{1: 0.154429, 2: 6e-05, 3: 0.601414}",2,0.000060,1,0.154429,0.154369


In [15]:
df[df['cell2mol'] == 9999]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
18,ABPYCU,Cu,2,9999,7777,"{1: 0.132927, 2: 6e-06, 3: 0.532555}",2,0.000006,1,0.132927,0.132921
43,ACMCUB,Cu,2,9999,7777,"{1: 0.164634, 2: 6e-05, 3: 9999}",2,0.000060,1,0.164634,0.164574
47,ACOXNI10,Cu,2,9999,7777,"{1: 0.146472, 2: 1.1e-05, 3: 0.503055}",2,0.000011,1,0.146472,0.146461
54,ACXACU,Cu,2,9999,7777,"{1: 0.207636, 2: 0.002269, 3: 0.654437}",2,0.002269,1,0.207636,0.205367
69,AEPCUI,Cu,2,9999,7777,"{1: 0.002664, 2: 0.014632, 3: 0.987621}",1,0.002664,2,0.014632,0.011968
...,...,...,...,...,...,...,...,...,...,...,...
11352,ZUYTUL,Cu,2,9999,7777,"{1: 0.254451, 2: 8.1e-05, 3: 0.471568}",2,0.000081,1,0.254451,0.254370
11353,ZUYVAT,Cu,2,9999,7777,"{1: 0.324099, 2: 0.000676, 3: 0.406939}",2,0.000676,1,0.324099,0.323423
11354,ZUYVIB,Cu,2,9999,7777,"{1: 0.267311, 2: 0.000353, 3: 0.451893}",2,0.000353,1,0.267311,0.266958
11355,ZUYVUN,Cu,2,9999,7777,"{1: 0.255744, 2: 0.000103, 3: 0.469588}",2,0.000103,1,0.255744,0.255641


# BVS

## Missing bond valence paramenters for all oxidation states

In [16]:
print(f"Total cases : {len(df)}")

# Missing bond valence paramenters for all oxidation states  ==> BVS code : 9999
print(f"Missing bond valence paramenters for all oxidation states : {len(df[(df['min'] == 9999.0)])}\n")
df[(df['min'] == 9999.0)] 

Total cases : 11361
Missing bond valence paramenters for all oxidation states : 75



,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
118,AGUNIA,Cu,2,9999,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN
223,AMICUW,Cu,1,9999,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN
226,AMIQOE,Cu,1,9999,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN
227,AMIRAR,Cu,1,9999,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN
450,AYAKOA,Cu,2,9999,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10155,WUKDIR,Cu,1,1,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN
10461,XIJXEU,Cu,1,1,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN
10996,YUDKUG,Cu,2,9999,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN
10997,YUDLAN,Cu,2,9999,9999,"{1: 9999, 2: 9999, 3: 9999}",1,9999.0,2,9999.0,NaN


## Only one OS available by BVS method

In [17]:
print(f"Only one OS available by BVS : {len( df[(df['min'] !=9999.0) & (df['second_min'] == 9999.0)] )}")
print(f"Only one OS available by BVS, delta >  0.5 : {len( df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0)])}")
print(f"Only one OS available by BVS, delta <=  0.5 : {len( df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0)])}")

Only one OS available by BVS : 33
Only one OS available by BVS, delta >  0.5 : 8
Only one OS available by BVS, delta <=  0.5 : 25


In [18]:
# Only one OS available by the bond-valence method
print(len(df[(df['min'] !=9999.0) & (df['second_min'] == 9999.0)]))
df[(df['min'] !=9999.0) & (df['second_min'] == 9999.0)]

33


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
58,ADEGEX,Cu,1,1,2,"{1: 9999, 2: 0.133133, 3: 9999}",2,0.133133,1,9999.0,NaN
181,AKONAQ,Cu,1,1,8888,"{1: 9999, 2: 0.508711, 3: 9999}",2,0.508711,1,9999.0,NaN
1247,CISCEN,Cu,1,1,8888,"{1: 9999, 2: 0.588902, 3: 9999}",2,0.588902,1,9999.0,NaN
1361,CTDGCU,Cu,2,2,2,"{1: 9999, 2: 0.056624, 3: 9999}",2,0.056624,1,9999.0,NaN
1667,DIJDIM,Cu,2,9999,2,"{1: 9999, 2: 0.000339, 3: 9999}",2,0.000339,1,9999.0,NaN
2143,EMUXIT,Cu,2,2,2,"{1: 9999, 2: 9.2e-05, 3: 9999}",2,0.000092,1,9999.0,NaN
3894,ILEBIL,Cu,2,2,2,"{1: 9999, 2: 7.6e-05, 3: 9999}",2,0.000076,1,9999.0,NaN
3920,IMIVAC,Cu,2,2,2,"{1: 9999, 2: 0.084876, 3: 9999}",2,0.084876,1,9999.0,NaN
4004,IREZUB,Cu,2,9999,2,"{1: 9999, 2: 0.000177, 3: 9999}",2,0.000177,1,9999.0,NaN
4309,JISBAO,Cu,1,1,2,"{1: 9999, 2: 0.009317, 3: 9999}",2,0.009317,1,9999.0,NaN


In [19]:
# Only one OS available, delta >  0.5  ==> BVS code : 8888
print(len(df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0)]))
df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0)]

8


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
181,AKONAQ,Cu,1,1,8888,"{1: 9999, 2: 0.508711, 3: 9999}",2,0.508711,1,9999.0,NaN
1247,CISCEN,Cu,1,1,8888,"{1: 9999, 2: 0.588902, 3: 9999}",2,0.588902,1,9999.0,NaN
4852,LADBUN,Cu,1,1,8888,"{1: 9999, 2: 0.607223, 3: 9999}",2,0.607223,1,9999.0,NaN
7352,QENLON,Cu,1,1,8888,"{1: 9999, 2: 0.552312, 3: 9999}",2,0.552312,1,9999.0,NaN
8127,SADZAA,Cu,2,9999,8888,"{1: 9999, 2: 0.948263, 3: 9999}",2,0.948263,1,9999.0,NaN
9769,VURGEW,Cu,1,1,8888,"{1: 9999, 2: 0.55682, 3: 9999}",2,0.556820,1,9999.0,NaN
10626,XUQFEU,Cu,2,9999,8888,"{1: 9999, 2: 1.931348, 3: 9999}",2,1.931348,1,9999.0,NaN
11072,ZAKFID,Cu,1,1,8888,"{1: 9999, 2: 0.589454, 3: 9999}",2,0.589454,1,9999.0,NaN


In [20]:
# Only one OS available, delta <= 0.5
print(len(df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0)]))
df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0)]

25


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
58,ADEGEX,Cu,1,1,2,"{1: 9999, 2: 0.133133, 3: 9999}",2,0.133133,1,9999.0,NaN
1361,CTDGCU,Cu,2,2,2,"{1: 9999, 2: 0.056624, 3: 9999}",2,0.056624,1,9999.0,NaN
1667,DIJDIM,Cu,2,9999,2,"{1: 9999, 2: 0.000339, 3: 9999}",2,0.000339,1,9999.0,NaN
2143,EMUXIT,Cu,2,2,2,"{1: 9999, 2: 9.2e-05, 3: 9999}",2,0.000092,1,9999.0,NaN
3894,ILEBIL,Cu,2,2,2,"{1: 9999, 2: 7.6e-05, 3: 9999}",2,0.000076,1,9999.0,NaN
3920,IMIVAC,Cu,2,2,2,"{1: 9999, 2: 0.084876, 3: 9999}",2,0.084876,1,9999.0,NaN
4004,IREZUB,Cu,2,9999,2,"{1: 9999, 2: 0.000177, 3: 9999}",2,0.000177,1,9999.0,NaN
4309,JISBAO,Cu,1,1,2,"{1: 9999, 2: 0.009317, 3: 9999}",2,0.009317,1,9999.0,NaN
4608,KENYUZ,Cu,2,2,2,"{1: 9999, 2: 0.249806, 3: 9999}",2,0.249806,1,9999.0,NaN
5071,LIBHOW,Cu,1,1,2,"{1: 9999, 2: 0.108565, 3: 9999}",2,0.108565,1,9999.0,NaN


## More than one OS available by BVS method

In [21]:
print(f"More than one OS available by BVS : {len(df[(df['min'] !=9999.0) & (df['second_min'] != 9999.0)])}")
print(f"OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5 : {len( df[(df['min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0)] )}")
print(f"OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 : {len( df[(df['min'] <= 0.5) & (df['second_min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0)] )}")
print(f"OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5 : {len( df[(df['min'] <= 0.5) & (df['second_min'] <= 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) ] )}")

More than one OS available by BVS : 11253
OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5 : 5
OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 : 292
OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5 : 10956


In [22]:
# More than one OS available by the bond-valence method
print(len(df[(df['min'] !=9999.0) & (df['second_min'] != 9999.0)]))
df[(df['min'] !=9999.0) & (df['second_min'] != 9999.0)]

11253


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.001990,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.284670
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
...,...,...,...,...,...,...,...,...,...,...,...
11355,ZUYVUN,Cu,2,9999,7777,"{1: 0.255744, 2: 0.000103, 3: 0.469588}",2,0.000103,1,0.255744,0.255641
11356,ZUYWAU,Cu,2,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2,0.000069,1,0.256987,0.256918
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


In [23]:
# OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5
print(len(df[(df['min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0)]))
df[(df['min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0)]

5


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
309,AROLOL,Cu,2,9999,8888,"{1: 1.000053, 2: 0.883316, 3: 1.025395}",2,0.883316,1,1.000053,NaN
2987,GERVAA,Cu,1,1,8888,"{1: 0.656007, 2: 1.278623, 3: 2.002267}",1,0.656007,2,1.278623,NaN
4208,JAYSIM,Cu,2,9999,8888,"{1: 0.766634, 2: 0.530514, 3: 0.617833}",2,0.530514,3,0.617833,NaN
7295,QAZNEM,Cu,2,2,8888,"{1: 0.907358, 2: 0.751686, 3: 0.8312}",2,0.751686,3,0.831200,NaN
9904,WEJMAC,Cu,2,2,8888,"{1: 1.060619, 2: 1.049258, 3: 0.847143}",3,0.847143,2,1.049258,NaN


In [24]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5
print(len(df[(df['min'] <= 0.5) & (df['second_min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0)]))
df[(df['min'] <= 0.5) & (df['second_min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0)]

292


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
97,AGEVUE,Cu,2,2,2,"{1: 0.566508, 2: 0.074936, 3: 9999}",2,0.074936,1,0.566508,NaN
137,AHUVUU,Cu,2,2,2,"{1: 0.537972, 2: 0.037138, 3: 9999}",2,0.037138,1,0.537972,NaN
140,AHUWIJ,Cu,2,2,2,"{1: 0.530102, 2: 0.026713, 3: 9999}",2,0.026713,1,0.530102,NaN
142,AHUWUV,Cu,2,2,2,"{1: 0.592179, 2: 0.052011, 3: 9999}",2,0.052011,1,0.592179,NaN
230,AMOGOA,Cu,1,1,1,"{1: 0.097754, 2: 0.614581, 3: 1.233498}",1,0.097754,2,0.614581,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11132,ZEDSOS,Cu,2,2,2,"{1: 0.552676, 2: 0.00177, 3: 0.863979}",2,0.001770,1,0.552676,NaN
11173,ZEYQEC,Cu,2,2,2,"{1: 0.569056, 2: 0.078311, 3: 9999}",2,0.078311,1,0.569056,NaN
11318,ZUDKUE,Cu,2,2,2,"{1: 0.550771, 2: 0.054091, 3: 9999}",2,0.054091,1,0.550771,NaN
11329,ZUJVUV,Cu,1,1,1,"{1: 0.192499, 2: 0.564125, 3: 9999}",1,0.192499,2,0.564125,NaN


In [25]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5
print(len(df[(df['min'] <= 0.5) & (df['second_min'] <= 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) ]))
df[(df['min'] <= 0.5) & (df['second_min'] <= 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) ]

10956


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.001990,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.284670
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
...,...,...,...,...,...,...,...,...,...,...,...
11355,ZUYVUN,Cu,2,9999,7777,"{1: 0.255744, 2: 0.000103, 3: 0.469588}",2,0.000103,1,0.255744,0.255641
11356,ZUYWAU,Cu,2,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2,0.000069,1,0.256987,0.256918
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


## BVS vs Cif

In [26]:
print(f"Same metal charge by BVS and cif : { len ( df[(df['Cif'] == df['bvs_min']) & (df['min'] != 9999.0)] ) }")
print(f"Different metal charge by BVS and cif : { len ( df[(df['Cif'] != df['bvs_min']) & (df['min'] != 9999.0)] ) }")

Same metal charge by BVS and cif : 10423
Different metal charge by BVS and cif : 863


In [27]:
# Same metal charge by BVS and cif
df[(df['Cif'] == df['bvs_min']) & (df['min'] != 9999.0)]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.001990,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.284670
5,ABEYAK,Cu,2,2,7777,"{1: 0.385019, 2: 0.03297, 3: 0.489935}",2,0.032970,1,0.385019,0.352049
6,ABEYIS,Cu,2,2,7777,"{1: 0.282888, 2: 0.116941, 3: 0.675027}",2,0.116941,1,0.282888,0.165947
...,...,...,...,...,...,...,...,...,...,...,...
11355,ZUYVUN,Cu,2,9999,7777,"{1: 0.255744, 2: 0.000103, 3: 0.469588}",2,0.000103,1,0.255744,0.255641
11356,ZUYWAU,Cu,2,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2,0.000069,1,0.256987,0.256918
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


In [28]:
# Different metal charge by BVS and cif
df[(df['Cif'] != df['bvs_min']) & (df['min'] != 9999.0)]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
44,ACMRCU10,Cu,2,2,7777,"{1: 0.03279, 2: 0.099746, 3: 0.918221}",1,0.032790,2,0.099746,0.066956
58,ADEGEX,Cu,1,1,2,"{1: 9999, 2: 0.133133, 3: 9999}",2,0.133133,1,9999.000000,NaN
69,AEPCUI,Cu,2,9999,7777,"{1: 0.002664, 2: 0.014632, 3: 0.987621}",1,0.002664,2,0.014632,0.011968
...,...,...,...,...,...,...,...,...,...,...,...
11282,ZOMSEB,Cu,2,2,7777,"{1: 0.507755, 2: 0.231143, 3: 0.154448}",3,0.154448,2,0.231143,0.076695
11285,ZONBOT,Cu,1,1,7777,"{1: 0.057526, 2: 0.000887, 3: 9999}",2,0.000887,1,0.057526,0.056639
11301,ZOWPIK,Cu,1,1,7777,"{1: 0.002448, 2: 0.002058, 3: 9999}",2,0.002058,1,0.002448,0.000390
11336,ZUPHEX,Cu,2,2,7777,"{1: 0.00622, 2: 0.073322, 3: 0.947664}",1,0.006220,2,0.073322,0.067102


In [29]:
print(f"Only one OS, BVS = cif : { len ( df[(df['min'] !=9999.0) & (df['second_min'] == 9999.0)& (df['Cif'] == df['bvs_min'])] ) }")
print(f"Only one OS, BVS != cif : { len ( df[(df['min'] !=9999.0) & (df['second_min'] == 9999.0)& (df['Cif'] != df['bvs_min'])] ) }")

print(f"Only one OS, delta >  0.5, BVS = cif : { len ( df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] == df['bvs_min'])] ) }")
print(f"Only one OS, delta >  0.5, BVS != cif : { len ( df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] != df['bvs_min'])] ) }")

print(f"Only one OS, delta <= 0.5, BVS = cif : { len ( df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] == df['bvs_min'])] ) }")
print(f"Only one OS, delta <= 0.5, BVS != cif : { len ( df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] != df['bvs_min'])] ) }")

Only one OS, BVS = cif : 19
Only one OS, BVS != cif : 14
Only one OS, delta >  0.5, BVS = cif : 2
Only one OS, delta >  0.5, BVS != cif : 6
Only one OS, delta <= 0.5, BVS = cif : 17
Only one OS, delta <= 0.5, BVS != cif : 8


In [30]:
# Only one OS available by the bond-valence method
print(len(df[(df['min'] !=9999.0) & (df['second_min'] == 9999.0)& (df['Cif'] == df['bvs_min'])]))
df[(df['min'] !=9999.0) & (df['second_min'] == 9999.0)& (df['Cif'] == df['bvs_min'])]

19


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
1361,CTDGCU,Cu,2,2,2,"{1: 9999, 2: 0.056624, 3: 9999}",2,0.056624,1,9999.0,NaN
1667,DIJDIM,Cu,2,9999,2,"{1: 9999, 2: 0.000339, 3: 9999}",2,0.000339,1,9999.0,NaN
2143,EMUXIT,Cu,2,2,2,"{1: 9999, 2: 9.2e-05, 3: 9999}",2,0.000092,1,9999.0,NaN
3894,ILEBIL,Cu,2,2,2,"{1: 9999, 2: 7.6e-05, 3: 9999}",2,0.000076,1,9999.0,NaN
3920,IMIVAC,Cu,2,2,2,"{1: 9999, 2: 0.084876, 3: 9999}",2,0.084876,1,9999.0,NaN
4004,IREZUB,Cu,2,9999,2,"{1: 9999, 2: 0.000177, 3: 9999}",2,0.000177,1,9999.0,NaN
4608,KENYUZ,Cu,2,2,2,"{1: 9999, 2: 0.249806, 3: 9999}",2,0.249806,1,9999.0,NaN
5240,LOXXAZ,Cu,2,2,2,"{1: 9999, 2: 0.010257, 3: 9999}",2,0.010257,1,9999.0,NaN
5440,MDZXCU,Cu,2,9999,2,"{1: 9999, 2: 4.3e-05, 3: 9999}",2,0.000043,1,9999.0,NaN
5470,MEJWAZ,Cu,2,2,2,"{1: 9999, 2: 0.124638, 3: 9999}",2,0.124638,1,9999.0,NaN


In [31]:
# Only one OS available, delta >  0.5  ==> BVS code : 8888 , BVS = cif
print(len(df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] == df['bvs_min'])]))
df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] == df['bvs_min'])]

2


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
8127,SADZAA,Cu,2,9999,8888,"{1: 9999, 2: 0.948263, 3: 9999}",2,0.948263,1,9999.0,NaN
10626,XUQFEU,Cu,2,9999,8888,"{1: 9999, 2: 1.931348, 3: 9999}",2,1.931348,1,9999.0,NaN


In [32]:
# Only one OS available, delta >  0.5  ==> BVS code : 8888 , BVS != cif
print(len(df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] != df['bvs_min'])]))
df[(df['min'] > 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] != df['bvs_min'])]

6


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
181,AKONAQ,Cu,1,1,8888,"{1: 9999, 2: 0.508711, 3: 9999}",2,0.508711,1,9999.0,NaN
1247,CISCEN,Cu,1,1,8888,"{1: 9999, 2: 0.588902, 3: 9999}",2,0.588902,1,9999.0,NaN
4852,LADBUN,Cu,1,1,8888,"{1: 9999, 2: 0.607223, 3: 9999}",2,0.607223,1,9999.0,NaN
7352,QENLON,Cu,1,1,8888,"{1: 9999, 2: 0.552312, 3: 9999}",2,0.552312,1,9999.0,NaN
9769,VURGEW,Cu,1,1,8888,"{1: 9999, 2: 0.55682, 3: 9999}",2,0.556820,1,9999.0,NaN
11072,ZAKFID,Cu,1,1,8888,"{1: 9999, 2: 0.589454, 3: 9999}",2,0.589454,1,9999.0,NaN


In [33]:
# Only one OS available, delta <= 0.5, BVS = cif
print(len(df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] == df['bvs_min'])]))
df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] == df['bvs_min'])]

17


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
1361,CTDGCU,Cu,2,2,2,"{1: 9999, 2: 0.056624, 3: 9999}",2,0.056624,1,9999.0,NaN
1667,DIJDIM,Cu,2,9999,2,"{1: 9999, 2: 0.000339, 3: 9999}",2,0.000339,1,9999.0,NaN
2143,EMUXIT,Cu,2,2,2,"{1: 9999, 2: 9.2e-05, 3: 9999}",2,0.000092,1,9999.0,NaN
3894,ILEBIL,Cu,2,2,2,"{1: 9999, 2: 7.6e-05, 3: 9999}",2,0.000076,1,9999.0,NaN
3920,IMIVAC,Cu,2,2,2,"{1: 9999, 2: 0.084876, 3: 9999}",2,0.084876,1,9999.0,NaN
4004,IREZUB,Cu,2,9999,2,"{1: 9999, 2: 0.000177, 3: 9999}",2,0.000177,1,9999.0,NaN
4608,KENYUZ,Cu,2,2,2,"{1: 9999, 2: 0.249806, 3: 9999}",2,0.249806,1,9999.0,NaN
5240,LOXXAZ,Cu,2,2,2,"{1: 9999, 2: 0.010257, 3: 9999}",2,0.010257,1,9999.0,NaN
5440,MDZXCU,Cu,2,9999,2,"{1: 9999, 2: 4.3e-05, 3: 9999}",2,0.000043,1,9999.0,NaN
5470,MEJWAZ,Cu,2,2,2,"{1: 9999, 2: 0.124638, 3: 9999}",2,0.124638,1,9999.0,NaN


In [34]:
# Only one OS available, delta <= 0.5, BVS != cif
print(len(df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] != df['bvs_min'])]))
df[(df['min'] <= 0.5) & (df['min'] != 9999.0) & (df['second_min'] == 9999.0) & (df['Cif'] != df['bvs_min'])]

8


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
58,ADEGEX,Cu,1,1,2,"{1: 9999, 2: 0.133133, 3: 9999}",2,0.133133,1,9999.0,NaN
4309,JISBAO,Cu,1,1,2,"{1: 9999, 2: 0.009317, 3: 9999}",2,0.009317,1,9999.0,NaN
5071,LIBHOW,Cu,1,1,2,"{1: 9999, 2: 0.108565, 3: 9999}",2,0.108565,1,9999.0,NaN
8662,TEFJIY,Cu,1,1,2,"{1: 9999, 2: 0.143038, 3: 9999}",2,0.143038,1,9999.0,NaN
9099,UDUSUH,Cu,1,9999,2,"{1: 9999, 2: 0.464975, 3: 9999}",2,0.464975,1,9999.0,NaN
9600,VIJFIF,Cu,1,1,2,"{1: 9999, 2: 0.384446, 3: 9999}",2,0.384446,1,9999.0,NaN
10927,YOGHIM,Cu,1,1,2,"{1: 9999, 2: 0.174313, 3: 9999}",2,0.174313,1,9999.0,NaN
11071,ZAKFEZ,Cu,1,1,2,"{1: 9999, 2: 0.499462, 3: 9999}",2,0.499462,1,9999.0,NaN


In [35]:
print(f"More than one OS, BVS = cif : { len( df[(df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] == df['bvs_min']) ] ) }")
print(f"More than one OS, BVS != cif : { len( df[(df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min']) ] ) }")
print("")
print(f"OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5, BVS = cif : { len( df[(df['min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] == df['bvs_min'])]) }")
print(f"OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5, BVS != cif : { len( df[(df['min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min'])]) }")
print(f"Another OS by BVS = cif : { len( df[(df['min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min']) & (df['Cif'] == df['bvs_second_min']) ]) }")
print(f"Another OS by BVS != cif : { len( df[(df['min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min']) & (df['Cif'] != df['bvs_second_min']) ]) }")

print("")
print(f"OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 , BVS = cif : { len(df[(df['min'] <= 0.5) & (df['second_min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] == df['bvs_min'])]) }")
print(f"OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 , BVS != cif : { len(df[(df['min'] <= 0.5) & (df['second_min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min'])]) }")
print(f"Another OS by BVS = cif : { len( df[(df['min'] <= 0.5) & (df['second_min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min']) & (df['Cif'] == df['bvs_second_min']) ]) }")
print(f"Another OS by BVS != cif : { len( df[(df['min'] <= 0.5) & (df['second_min'] > 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min']) & (df['Cif'] != df['bvs_second_min']) ]) }")


print("")
print(f"OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5, BVS = cif : { len(df[(df['min'] <= 0.5) & (df['second_min'] <= 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] == df['bvs_min'])]) }")
print(f"OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5, BVS != cif : { len(df[(df['min'] <= 0.5) & (df['second_min'] <= 0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min'])]) }")
print(f"Another OS by BVS = cif : { len( df[(df['min'] <= 0.5) & (df['second_min'] <=  0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min']) & (df['Cif'] == df['bvs_second_min']) ]) }")
print(f"Another OS by BVS != cif : { len( df[(df['min'] <= 0.5) & (df['second_min'] <=  0.5) & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] != df['bvs_min']) & (df['Cif'] != df['bvs_second_min']) ]) }")

More than one OS, BVS = cif : 10404
More than one OS, BVS != cif : 849

OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5, BVS = cif : 4
OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5, BVS != cif : 1
Another OS by BVS = cif : 1
Another OS by BVS != cif : 0

OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 , BVS = cif : 273
OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 , BVS != cif : 19
Another OS by BVS = cif : 15
Another OS by BVS != cif : 4

OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5, BVS = cif : 10127
OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5, BVS != cif : 829
Another OS by BVS = cif : 785
Another OS by BVS != cif : 44


In [36]:
# More than one OS available by the bond-valence method
print(len(df[(df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] == df['bvs_min']) ]))
df[(df['min'] !=9999.0) & (df['second_min'] != 9999.0) & (df['Cif'] == df['bvs_min']) ]

10404


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.001990,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.284670
5,ABEYAK,Cu,2,2,7777,"{1: 0.385019, 2: 0.03297, 3: 0.489935}",2,0.032970,1,0.385019,0.352049
6,ABEYIS,Cu,2,2,7777,"{1: 0.282888, 2: 0.116941, 3: 0.675027}",2,0.116941,1,0.282888,0.165947
...,...,...,...,...,...,...,...,...,...,...,...
11355,ZUYVUN,Cu,2,9999,7777,"{1: 0.255744, 2: 0.000103, 3: 0.469588}",2,0.000103,1,0.255744,0.255641
11356,ZUYWAU,Cu,2,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2,0.000069,1,0.256987,0.256918
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


In [37]:
#OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5, BVS = cif 
print(len(df[(df['min'] > 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0)
             & (df['Cif'] == df['bvs_min']) ]))
df[(df['min'] > 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] == df['bvs_min']) ]

4


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
309,AROLOL,Cu,2,9999,8888,"{1: 1.000053, 2: 0.883316, 3: 1.025395}",2,0.883316,1,1.000053,NaN
2987,GERVAA,Cu,1,1,8888,"{1: 0.656007, 2: 1.278623, 3: 2.002267}",1,0.656007,2,1.278623,NaN
4208,JAYSIM,Cu,2,9999,8888,"{1: 0.766634, 2: 0.530514, 3: 0.617833}",2,0.530514,3,0.617833,NaN
7295,QAZNEM,Cu,2,2,8888,"{1: 0.907358, 2: 0.751686, 3: 0.8312}",2,0.751686,3,0.831200,NaN


In [38]:
#OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5, BVS != cif 
print(len(df[(df['min'] > 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0)
             & (df['Cif'] != df['bvs_min']) ]))
df[(df['min'] > 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min']) ]

1


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
9904,WEJMAC,Cu,2,2,8888,"{1: 1.060619, 2: 1.049258, 3: 0.847143}",3,0.847143,2,1.049258,NaN


In [39]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 , BVS = cif
print(len(df[(df['min'] <= 0.5) 
             & (df['second_min'] > 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0) 
            & (df['Cif'] == df['bvs_min'])]))
df[(df['min'] <= 0.5) 
   & (df['second_min'] > 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] == df['bvs_min'])]

273


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
97,AGEVUE,Cu,2,2,2,"{1: 0.566508, 2: 0.074936, 3: 9999}",2,0.074936,1,0.566508,NaN
137,AHUVUU,Cu,2,2,2,"{1: 0.537972, 2: 0.037138, 3: 9999}",2,0.037138,1,0.537972,NaN
140,AHUWIJ,Cu,2,2,2,"{1: 0.530102, 2: 0.026713, 3: 9999}",2,0.026713,1,0.530102,NaN
142,AHUWUV,Cu,2,2,2,"{1: 0.592179, 2: 0.052011, 3: 9999}",2,0.052011,1,0.592179,NaN
230,AMOGOA,Cu,1,1,1,"{1: 0.097754, 2: 0.614581, 3: 1.233498}",1,0.097754,2,0.614581,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11132,ZEDSOS,Cu,2,2,2,"{1: 0.552676, 2: 0.00177, 3: 0.863979}",2,0.001770,1,0.552676,NaN
11173,ZEYQEC,Cu,2,2,2,"{1: 0.569056, 2: 0.078311, 3: 9999}",2,0.078311,1,0.569056,NaN
11318,ZUDKUE,Cu,2,2,2,"{1: 0.550771, 2: 0.054091, 3: 9999}",2,0.054091,1,0.550771,NaN
11329,ZUJVUV,Cu,1,1,1,"{1: 0.192499, 2: 0.564125, 3: 9999}",1,0.192499,2,0.564125,NaN


In [40]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 , BVS != cif
print(len(df[(df['min'] <= 0.5) 
             & (df['second_min'] > 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0) 
            & (df['Cif'] != df['bvs_min'])]))
df[(df['min'] <= 0.5) 
   & (df['second_min'] > 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min'])]

19


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
632,BEJKUZ,Cu,2,2,1,"{1: 0.012051, 2: 0.670008, 3: 1.357887}",1,0.012051,2,0.670008,NaN
1019,CAKTUF,Cu,2,2,1,"{1: 0.006961, 2: 0.532982, 3: 1.287711}",1,0.006961,2,0.532982,NaN
1156,CETDCU10,Cu,2,2,1,"{1: 0.001795, 2: 0.501414, 3: 1.198742}",1,0.001795,2,0.501414,NaN
1730,DODPES,Cu,3,9999,2,"{1: 0.569248, 2: 0.036431, 3: 9999}",2,0.036431,1,0.569248,NaN
2004,EFOWIH,Cu,2,2,1,"{1: 0.024584, 2: 0.550702, 3: 9999}",1,0.024584,2,0.550702,NaN
2801,FULYAN,Cu,2,2,1,"{1: 0.018978, 2: 0.512764, 3: 9999}",1,0.018978,2,0.512764,NaN
2803,FULYIV,Cu,2,2,1,"{1: 0.208873, 2: 0.880475, 3: 9999}",1,0.208873,2,0.880475,NaN
2804,FULYOB,Cu,2,2,1,"{1: 0.001066, 2: 0.560318, 3: 1.416695}",1,0.001066,2,0.560318,NaN
3035,GICNIP,Cu,2,2,1,"{1: 0.0094, 2: 0.506695, 3: 9999}",1,0.009400,2,0.506695,NaN
3040,GIDCIF01,Cu,2,2,1,"{1: 0.225288, 2: 0.90102, 3: 9999}",1,0.225288,2,0.901020,NaN


In [41]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5, BVS = cif
print(len(df[(df['min'] <= 0.5) 
             & (df['second_min']  <= 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0) 
             & (df['Cif'] == df['bvs_min'])]))
df[(df['min'] <= 0.5) & (df['second_min']  <=0.5) 
   & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) 
   & (df['Cif'] == df['bvs_min'])]

10127


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABAFAN,Cu,1,1,7777,"{1: 0.00199, 2: 0.015306, 3: 0.988304}",1,0.001990,2,0.015306,0.013316
1,ABAFER,Cu,1,1,7777,"{1: 0.001237, 2: 0.132774, 3: 1.10737}",1,0.001237,2,0.132774,0.131537
2,ABAMUO,Cu,2,2,7777,"{1: 0.284689, 2: 1.9e-05, 3: 0.482799}",2,0.000019,1,0.284689,0.284670
5,ABEYAK,Cu,2,2,7777,"{1: 0.385019, 2: 0.03297, 3: 0.489935}",2,0.032970,1,0.385019,0.352049
6,ABEYIS,Cu,2,2,7777,"{1: 0.282888, 2: 0.116941, 3: 0.675027}",2,0.116941,1,0.282888,0.165947
...,...,...,...,...,...,...,...,...,...,...,...
11355,ZUYVUN,Cu,2,9999,7777,"{1: 0.255744, 2: 0.000103, 3: 0.469588}",2,0.000103,1,0.255744,0.255641
11356,ZUYWAU,Cu,2,9999,7777,"{1: 0.256987, 2: 6.9e-05, 3: 0.443784}",2,0.000069,1,0.256987,0.256918
11357,ZUZKAG,Cu,2,2,7777,"{1: 0.206968, 2: 7.6e-05, 3: 0.461207}",2,0.000076,1,0.206968,0.206892
11359,ZUZZEC,Cu,2,2,7777,"{1: 0.236939, 2: 8.2e-05, 3: 0.492313}",2,0.000082,1,0.236939,0.236857


In [42]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5, BVS != cif
print(len(df[(df['min'] <= 0.5) 
             & (df['second_min']  <= 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0) 
             & (df['Cif'] != df['bvs_min'])]))
df[(df['min'] <= 0.5) & (df['second_min']  <=0.5) 
   & (df['min'] !=9999.0) & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min'])]

829


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
44,ACMRCU10,Cu,2,2,7777,"{1: 0.03279, 2: 0.099746, 3: 0.918221}",1,0.032790,2,0.099746,0.066956
69,AEPCUI,Cu,2,9999,7777,"{1: 0.002664, 2: 0.014632, 3: 0.987621}",1,0.002664,2,0.014632,0.011968
88,AFOLOW,Cu,2,2,7777,"{1: 0.557945, 2: 0.286795, 3: 0.176542}",3,0.176542,2,0.286795,0.110253
...,...,...,...,...,...,...,...,...,...,...,...
11282,ZOMSEB,Cu,2,2,7777,"{1: 0.507755, 2: 0.231143, 3: 0.154448}",3,0.154448,2,0.231143,0.076695
11285,ZONBOT,Cu,1,1,7777,"{1: 0.057526, 2: 0.000887, 3: 9999}",2,0.000887,1,0.057526,0.056639
11301,ZOWPIK,Cu,1,1,7777,"{1: 0.002448, 2: 0.002058, 3: 9999}",2,0.002058,1,0.002448,0.000390
11336,ZUPHEX,Cu,2,2,7777,"{1: 0.00622, 2: 0.073322, 3: 0.947664}",1,0.006220,2,0.073322,0.067102


## Another OS

In [43]:
# OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5, BVS != cif , Another OS by BVS = cif
print(len(df[(df['min'] > 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0)
             & (df['Cif'] != df['bvs_min']) 
             & (df['Cif'] == df['bvs_second_min'])]))
df[(df['min'] > 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min'])
   & (df['Cif'] == df['bvs_second_min'])]

1


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
9904,WEJMAC,Cu,2,2,8888,"{1: 1.060619, 2: 1.049258, 3: 0.847143}",3,0.847143,2,1.049258,NaN


In [44]:
# OS with smallest Δ > 0.5, Another OS with 2nd smallest Δ > 0.5, BVS != cif , Another OS by BVS != cif
print(len(df[(df['min'] > 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0)
             & (df['Cif'] != df['bvs_min']) 
             & (df['Cif'] != df['bvs_second_min'])]))
df[(df['min'] > 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min'])
   & (df['Cif'] != df['bvs_second_min'])]

0


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff


In [45]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 , BVS != cif, Another OS by BVS = cif
print(len(df[(df['min'] <= 0.5) 
             & (df['second_min'] > 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0) 
             & (df['Cif'] != df['bvs_min'])
             & (df['Cif'] == df['bvs_second_min'])]))
df[(df['min'] <= 0.5) 
   & (df['second_min'] > 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min']) 
   & (df['Cif'] == df['bvs_second_min'])]

15


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
632,BEJKUZ,Cu,2,2,1,"{1: 0.012051, 2: 0.670008, 3: 1.357887}",1,0.012051,2,0.670008,NaN
1019,CAKTUF,Cu,2,2,1,"{1: 0.006961, 2: 0.532982, 3: 1.287711}",1,0.006961,2,0.532982,NaN
1156,CETDCU10,Cu,2,2,1,"{1: 0.001795, 2: 0.501414, 3: 1.198742}",1,0.001795,2,0.501414,NaN
2004,EFOWIH,Cu,2,2,1,"{1: 0.024584, 2: 0.550702, 3: 9999}",1,0.024584,2,0.550702,NaN
2801,FULYAN,Cu,2,2,1,"{1: 0.018978, 2: 0.512764, 3: 9999}",1,0.018978,2,0.512764,NaN
2803,FULYIV,Cu,2,2,1,"{1: 0.208873, 2: 0.880475, 3: 9999}",1,0.208873,2,0.880475,NaN
2804,FULYOB,Cu,2,2,1,"{1: 0.001066, 2: 0.560318, 3: 1.416695}",1,0.001066,2,0.560318,NaN
3035,GICNIP,Cu,2,2,1,"{1: 0.0094, 2: 0.506695, 3: 9999}",1,0.009400,2,0.506695,NaN
3040,GIDCIF01,Cu,2,2,1,"{1: 0.225288, 2: 0.90102, 3: 9999}",1,0.225288,2,0.901020,NaN
4638,KESYUC,Cu,2,2,1,"{1: 0.000202, 2: 0.542374, 3: 1.22443}",1,0.000202,2,0.542374,NaN


In [46]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ > 0.5 , BVS != cif, Another OS by BVS != cif
print(len(df[(df['min'] <= 0.5) 
             & (df['second_min'] > 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0) 
             & (df['Cif'] != df['bvs_min'])
             & (df['Cif'] != df['bvs_second_min'])]))
df[(df['min'] <= 0.5) 
   & (df['second_min'] > 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min']) 
   & (df['Cif'] != df['bvs_second_min'])]

4


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
1730,DODPES,Cu,3,9999,2,"{1: 0.569248, 2: 0.036431, 3: 9999}",2,0.036431,1,0.569248,NaN
7221,QAKWOP,Cu,3,9999,2,"{1: 0.518773, 2: 0.013621, 3: 9999}",2,0.013621,1,0.518773,NaN
9129,UGASOL,Cu,3,1,2,"{1: 0.522992, 2: 0.062375, 3: 9999}",2,0.062375,1,0.522992,NaN
9748,VULFEQ,Cu,3,9999,2,"{1: 0.514304, 2: 0.001925, 3: 9999}",2,0.001925,1,0.514304,NaN


In [47]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5, BVS != cif , Another OS by BVS = cif
print(len(df[(df['min'] <= 0.5) 
             & (df['second_min'] <= 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0) 
             & (df['Cif'] != df['bvs_min'])
             & (df['Cif'] == df['bvs_second_min'])]))
df[(df['min'] <= 0.5) 
   & (df['second_min'] <= 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min']) 
   & (df['Cif'] == df['bvs_second_min'])]

785


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
3,ABETEH,Cu,1,1,7777,"{1: 0.002921, 2: 0.001846, 3: 9999}",2,0.001846,1,0.002921,0.001075
4,ABETIL,Cu,1,1,7777,"{1: 0.019641, 2: 0.000227, 3: 9999}",2,0.000227,1,0.019641,0.019414
44,ACMRCU10,Cu,2,2,7777,"{1: 0.03279, 2: 0.099746, 3: 0.918221}",1,0.032790,2,0.099746,0.066956
69,AEPCUI,Cu,2,9999,7777,"{1: 0.002664, 2: 0.014632, 3: 0.987621}",1,0.002664,2,0.014632,0.011968
88,AFOLOW,Cu,2,2,7777,"{1: 0.557945, 2: 0.286795, 3: 0.176542}",3,0.176542,2,0.286795,0.110253
...,...,...,...,...,...,...,...,...,...,...,...
11282,ZOMSEB,Cu,2,2,7777,"{1: 0.507755, 2: 0.231143, 3: 0.154448}",3,0.154448,2,0.231143,0.076695
11285,ZONBOT,Cu,1,1,7777,"{1: 0.057526, 2: 0.000887, 3: 9999}",2,0.000887,1,0.057526,0.056639
11301,ZOWPIK,Cu,1,1,7777,"{1: 0.002448, 2: 0.002058, 3: 9999}",2,0.002058,1,0.002448,0.000390
11336,ZUPHEX,Cu,2,2,7777,"{1: 0.00622, 2: 0.073322, 3: 0.947664}",1,0.006220,2,0.073322,0.067102


In [48]:
# OS with smallest Δ ≤ 0.5, Another OS with 2nd smallest Δ ≤ 0.5, BVS != cif , Another OS by BVS != cif
print(len(df[(df['min'] <= 0.5) 
             & (df['second_min'] <= 0.5) 
             & (df['min'] !=9999.0) 
             & (df['second_min'] != 9999.0) 
             & (df['Cif'] != df['bvs_min'])
             & (df['Cif'] != df['bvs_second_min'])]))
df[(df['min'] <= 0.5) 
   & (df['second_min'] <= 0.5) 
   & (df['min'] !=9999.0) 
   & (df['second_min'] != 9999.0) 
   & (df['Cif'] != df['bvs_min']) 
   & (df['Cif'] != df['bvs_second_min'])]

44


,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
177,AKIYUO,Cu,3,2,7777,"{1: 0.257117, 2: 5.8e-05, 3: 0.49775}",2,0.000058,1,0.257117,0.257059
423,AWOGEY,Cu,3,9999,7777,"{1: 0.488264, 2: 0.001882, 3: 9999}",2,0.001882,1,0.488264,0.486382
763,BIPSUO,Cu,0,2,7777,"{1: 0.088705, 2: 0.000434, 3: 0.751827}",2,0.000434,1,0.088705,0.088271
955,BUWHAD,Cu,3,9999,7777,"{1: 0.17302, 2: 0.001001, 3: 9999}",2,0.001001,1,0.173020,0.172019
1833,DUFPOK,Cu,3,1,7777,"{1: 0.442011, 2: 0.05538, 3: 9999}",2,0.055380,1,0.442011,0.386631
1931,ECAHUO,Cu,3,9999,7777,"{1: 0.496017, 2: 6.2e-05, 3: 9999}",2,0.000062,1,0.496017,0.495955
3366,HEBSOX,Cu,3,9999,7777,"{1: 0.430327, 2: 0.073323, 3: 9999}",2,0.073323,1,0.430327,0.357004
3367,HEBSUD,Cu,3,9999,7777,"{1: 0.470358, 2: 0.018243, 3: 9999}",2,0.018243,1,0.470358,0.452115
3716,IBESAJ,Cu,3,9999,7777,"{1: 0.275656, 2: 0.016702, 3: 9999}",2,0.016702,1,0.275656,0.258954
3832,IGUBET,Cu,3,9999,7777,"{1: 0.426961, 2: 0.015791, 3: 9999}",2,0.015791,1,0.426961,0.411170
